In [1]:
import torch
import ssl
import matplotlib.pyplot as plt
from src.logger.logger_initializer import LoggerInitializer

ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__

LoggerInitializer().init()

[2023-04-17 02:02:37] 21831 root {logger_initializer-47} INFO - Current process ID: 21831


True

In [2]:
from src.data_loader.dataset_getter import DatasetGetter
dataset = DatasetGetter('../../bayesian_test/data', batch_size=12).get()

In [3]:
from src.models.bnn.pyro_resnet18_bnn_classifier import PyroResnet18BnnClassifier

classifier = PyroResnet18BnnClassifier(dataset=dataset, device=device).init()

In [ ]:
classifier.freeze_all_layers_except_fc()
result_df, result_dict = classifier.fit(num_epoch=300)
display(result_df)
print(result_dict)

[2023-04-17 02:03:34] 21831 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 0/300]: f1_score: 0.127 accuracy_score: 0.230 loss: 1115049189189214.875 
[2023-04-17 02:04:06] 21831 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 1/300]: f1_score: 0.141 accuracy_score: 0.211 loss: 844162542401496.000 
[2023-04-17 02:04:28] 21831 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 2/300]: f1_score: 0.107 accuracy_score: 0.199 loss: 621173413055837.125 
[2023-04-17 02:04:50] 21831 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 3/300]: f1_score: 0.174 accuracy_score: 0.246 loss: 441415407054063.500 
[2023-04-17 02:05:44] 21831 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 4/300]: f1_score: 0.236 accuracy_score: 0.309 loss: 300223850080617.562 
[2023-04-17 02:06:48] 21831 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 5/300]: f1_score: 0.261 accuracy_score: 

In [ ]:
classifier.save_model(result_dict)

In [6]:
test_model_weights_path = '../../bayesian_refactoring/assets/model_weights/pyro_resnet18bnn_28766_97cd2c3b-14db-49e6-9035-0474b00c51be.pt'

In [7]:
test_results = torch.load(test_model_weights_path)

In [8]:
new_classifier = PyroResnet18BnnClassifier(
    dataset=dataset, device=device
).load_model(
    model_state_dict=test_results['best_f1_score_model'],
    pyro_state_dict=test_results['pyro_state_dict']
)

In [9]:
new_classifier.score(dataset.validate_dataloader)

{'f1_score': 0.11227969435503775, 'accuracy_score': 0.22924901185770752}